# Broadcast with Quantum Teleportation
แทนที่ CNOT gates ด้วย quantum teleportation protocol

In [13]:
import importlib
import MPI_collective
import MPI
import qiskit
import math
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
importlib.reload(MPI_collective)
importlib.reload(MPI)

<module 'MPI' from '/workspaces/qmpi_cpe_kmutt/QMPI_Tinthai_Version/playground/collective_optimization/MPI.py'>

In [14]:
# Quantum Teleportation Functions
def quantum_teleportation(circuit, source_qubit, target_qubit, ancilla_qubit, classical_bits):
    """
    Quantum Teleportation Protocol
    """
    # Step 1: สร้าง Bell pair ระหว่าง ancilla และ target
    circuit.h(ancilla_qubit)
    circuit.cx(ancilla_qubit, target_qubit)
    
    # Step 2: Bell measurement ระหว่าง source และ ancilla
    circuit.cx(source_qubit, ancilla_qubit)
    circuit.h(source_qubit)
    
    # Step 3: วัด source และ ancilla qubits
    circuit.measure(source_qubit, classical_bits[0])
    circuit.measure(ancilla_qubit, classical_bits[1])
    
    # Step 4: Conditional operations บน target qubit
    circuit.cx(ancilla_qubit, target_qubit)  # ถ้า ancilla = 1
    circuit.cz(source_qubit, target_qubit)   # ถ้า source = 1

def add_teleportation_to_circuit(circuit, epr_qubit, target_qubit, node_name):
    """
    เพิ่ม teleportation protocol ลงใน circuit
    """
    # เพิ่ม ancilla qubit
    ancilla = QuantumRegister(1, name=f"ancilla_{node_name}")
    circuit.add_register(ancilla)
    
    # เพิ่ม classical register สำหรับการวัด
    classical_reg = ClassicalRegister(2, name=f"c_{node_name}")
    circuit.add_register(classical_reg)
    
    # ทำ quantum teleportation
    quantum_teleportation(circuit, epr_qubit, target_qubit, ancilla[0], classical_reg)
    
    return ancilla, classical_reg

In [15]:
# สร้าง circuit เหมือนเดิม
collective_circuit = QuantumCircuit()
nodes_count = 5
qnode = {}
for i in range(nodes_count):
    qnode[i] = QuantumRegister(1, name = f"node{i}")
    collective_circuit.add_register(qnode[i])

In [16]:
# Initiate ให้ node0[0] มีโอกาส ket(0) ~85%
collective_circuit.rx(math.pi/4, qnode[0][0])

collective_class = MPI_collective.CollectiveMPI(collective_circuit)

# Broadcast operation
operation = collective_class.brodcast([qnode[0][0], qnode[1][0], qnode[2][0], qnode[3][0], qnode[4][0]])

In [ ]:
# สมมติว่าเรามี EPR qubits สำหรับ node 1, 2, 3
# (ในการใช้งานจริงจะต้องสร้างจาก broadcast operation)

# สร้าง EPR registers
node1_epr = QuantumRegister(1, name="node1_epr")
node2_epr = QuantumRegister(1, name="node2_epr")
node3_epr = QuantumRegister(1, name="node3_epr")

collective_circuit.add_register(node1_epr)
collective_circuit.add_register(node2_epr)
collective_circuit.add_register(node3_epr)

# แทนที่ CNOT gates ด้วย quantum teleportation
print("แทนที่ CNOT gates ด้วย quantum teleportation...")

# แทนที่: collective_circuit.cx(node1_epr[0], qnode[1][0])
ancilla1, c1 = add_teleportation_to_circuit(collective_circuit, node1_epr[0], qnode[1][0], "node1")

# แทนที่: collective_circuit.cx(node2_epr[0], qnode[2][0])
ancilla2, c2 = add_teleportation_to_circuit(collective_circuit, node2_epr[0], qnode[2][0], "node2")

# แทนที่: collective_circuit.cx(node3_epr[0], qnode[3][0])
ancilla3, c3 = add_teleportation_to_circuit(collective_circuit, node3_epr[0], qnode[3][0], "node3")

print("เสร็จสิ้นการแทนที่ด้วย quantum teleportation!")

แทนที่ CNOT gates ด้วย quantum teleportation...


CircuitError: 'Bit \'<Qubit register=(1, "node1_epr"), index=0>\' is not in the circuit.'

In [ ]:
# แสดง circuit ที่ได้
print(f"จำนวน qubits ทั้งหมด: {collective_circuit.num_qubits}")
print(f"จำนวน classical bits ทั้งหมด: {collective_circuit.num_clbits}")
print(f"ความลึกของ circuit: {collective_circuit.depth()}")

# วาด circuit (ถ้าไม่ใหญ่เกินไป)
if collective_circuit.num_qubits <= 15:
    print(collective_circuit.draw())
else:
    print("Circuit ใหญ่เกินไปที่จะแสดง")

In [ ]:
# ทดสอบ circuit ด้วย simulator
simulator = AerSimulator()

# Transpile circuit
transpiled_circuit = transpile(collective_circuit, simulator)

# รัน simulation
job = simulator.run(transpiled_circuit, shots=1024)
result = job.result()
counts = result.get_counts()

print("ผลการ simulation:")
for state, count in sorted(counts.items()):
    print(f"{state}: {count}")

## สรุปการเปลี่ยนแปลง

### เดิม (ใช้ CNOT gates):
```python
collective_circuit_tinthai.q.cx(node1_epr[0], node1[0])
collective_circuit_tinthai.q.cx(node2_epr[0], node2[0])
collective_circuit_tinthai.q.cx(node3_epr[0], node3[0])
```

### ใหม่ (ใช้ Quantum Teleportation):
```python
# แทนที่ด้วย quantum teleportation protocol
ancilla1, c1 = add_teleportation_to_circuit(collective_circuit, node1_epr[0], qnode[1][0], "node1")
ancilla2, c2 = add_teleportation_to_circuit(collective_circuit, node2_epr[0], qnode[2][0], "node2")
ancilla3, c3 = add_teleportation_to_circuit(collective_circuit, node3_epr[0], qnode[3][0], "node3")
```

### ข้อดีของ Quantum Teleportation:
1. **ความปลอดภัย**: ไม่สามารถดักฟังข้อมูลได้โดยไม่ถูกตรวจพบ
2. **การรักษา Quantum State**: สามารถส่ง quantum state ที่ไม่ทราบค่าได้อย่างสมบูรณ์
3. **No-Cloning Theorem**: ป้องกันการคัดลอก quantum information
4. **Distributed Computing**: เหมาะสำหรับ quantum network และ distributed quantum computing
